In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout

warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
#import datesets
train = pd.read_csv('../input/tabular-playground-series-apr-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-apr-2021/test.csv')

In [ ]:
train.head()

# EDA

In [ ]:
plt.figure(figsize=(10,6))
sns.set_style('whitegrid')
sns.countplot(x='Survived',data=train,palette='coolwarm')

In [ ]:
plt.figure(figsize=(10,6))
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Sex',data=train,palette='coolwarm')

In [ ]:
plt.figure(figsize=(10,6))
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Pclass',data=train,palette='rainbow')

In [ ]:
plt.figure(figsize=(10,6))
train['Age'].hist(bins=30,color='blue')

In [ ]:
plt.figure(figsize=(10, 7))
sns.boxplot(x='Pclass',y='Age',data=train,palette='rainbow')

In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(train['Fare'].dropna(),color='darkred',bins=30)

**Missing Values**

In [ ]:
#Missing data check
plt.figure(figsize=(10,6))
sns.heatmap(train.isnull(),cmap='viridis')

test.head()

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(test.isnull(),cmap='viridis')

In [ ]:
print(f"Average Age of Pclass 1 passengers {train[train['Pclass']==1]['Age'].mean():.2f}")

print(f"Average Age of Pclass 2 passengers {train[train['Pclass']==2]['Age'].mean():.2f}")

print(f"Average Age of Pclass 3 passengers {train[train['Pclass']==3]['Age'].mean():.2f}")


In [ ]:
#filling missing values for Age based on the Pclass
def fill_age(c):
    Age = c[0]
    Pclass = c[1]
    
    if pd.isnull(Age):
        if Pclass == 1:
            return 44
        elif Pclass == 2:
            return 38
        else:
            return 33
    else:
        return Age

train['Age'] = train[['Age','Pclass']].apply(fill_age,axis=1)
test['Age'] = train[['Age','Pclass']].apply(fill_age,axis=1)

In [ ]:
train.drop(['Ticket','Cabin'],axis=1,inplace=True)
test.drop(['Ticket','Cabin'],axis=1,inplace=True)

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(train.isnull(),cmap='viridis')

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(test.isnull(),cmap='viridis')

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
train[['Fare']] = imputer.fit_transform(train[['Fare']])
test[['Fare']] = imputer.fit_transform(test[['Fare']])

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
train[['Embarked']] = imputer.fit_transform(train[['Embarked']])
test[['Embarked']] = imputer.fit_transform(test[['Embarked']])

In [ ]:
sns.heatmap(test.isnull(),cmap='viridis')

In [ ]:
sns.heatmap(test.isnull(),cmap='viridis')

**Encoding Categroical Data**

In [ ]:
print(train.info())
print(test.info())

In [ ]:
#train data
S = pd.get_dummies(train['Sex'],drop_first=True)
E = pd.get_dummies(train['Embarked'],drop_first=True)
train = pd.concat([train,S,E],axis=1)
train.drop(['Sex','Embarked','Name'],axis=1,inplace=True)

In [ ]:
#test data
S = pd.get_dummies(test['Sex'],drop_first=True)
E = pd.get_dummies(test['Embarked'],drop_first=True)
test = pd.concat([test,S,E],axis=1)
test.drop(['Sex','Embarked','Name'],axis=1,inplace=True)


In [ ]:
train.head()

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(train.corr(),cmap='rainbow')

In [ ]:
train.corr()['Survived']

In [ ]:
test.head()

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
X = train.drop(['PassengerId','Survived'],axis=1)
y = train['Survived']

In [ ]:
test

**Data Scaling**

In [ ]:
sc = StandardScaler()
X = sc.fit_transform(X)
#test data
scaled_test = sc.transform(test.drop('PassengerId',axis=1))

In [ ]:
X

In [ ]:
scaled_test

**Splitting the train data for model building**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30,random_state=100)

In [ ]:
print(X_train.shape)
print(X_test.shape)

# Logistic Regression Model

In [ ]:
lr = LogisticRegression()
lr.fit(X_train,y_train)
predictions = lr.predict(X_test)

**Evaluation**

In [ ]:
print(f'Classification Report\n {classification_report(y_test,predictions)}')
print(f'Confusion Matrix \n {confusion_matrix(y_test,predictions)}')
print(f'\nAccuracy: {accuracy_score(y_test,predictions)*100:.2f}%')


In [ ]:
#fitting the whole train data and predciting results for test data
lr.fit(X,y)
lr_prediction = lr.predict(scaled_test)

In [ ]:
submission = pd.DataFrame({'PassengerId': test['PassengerId'],'Survived':lr_prediction.flatten()}) 
submission.to_csv('submission.csv',index=False)

# Random Forest Classifier

In [ ]:
rf = RandomForestClassifier(n_estimators = 200, criterion = 'entropy', random_state = 0)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

In [ ]:
print(f'Classification Report\n {classification_report(y_test,y_pred)}')
print(f'Confusion Matrix \n {confusion_matrix(y_test,y_pred)}')
print(f'\nAccuracy: {accuracy_score(y_test,y_pred)*100:.2f}%')


In [ ]:
#fitting the whole train data and predciting results for test data
rf.fit(X,y)
rf_prediction = rf.predict(scaled_test)

In [ ]:
submission = pd.DataFrame({'PassengerId': test['PassengerId'],'Survived':rf_prediction.flatten()}) 
submission.to_csv('submission.csv',index=False)

# ANN

In [ ]:
model = Sequential()
model.add(Dense(units=30,activation='relu'))
model.add(Dense(units=15,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam')

In [ ]:
model.fit(x=X_train,y=y_train,epochs=100,validation_data=(X_test, y_test), verbose=1)

In [ ]:
model_loss = pd.DataFrame(model.history.history)
model_loss.plot()

In [ ]:
tf_prediction = model.predict_classes(X_test)

In [ ]:
print(f'Classification Report\n {classification_report(y_test,tf_prediction)}')
print(f'Confusion Matrix \n {confusion_matrix(y_test,tf_prediction)}')
print(f'\nAccuracy: {accuracy_score(y_test,tf_prediction)*100:.2f}%')


In [ ]:
#Whole train dataset
model.fit(x=X,y=y,epochs=100,verbose=1)

In [ ]:
tf_prediction = model.predict_classes(scaled_test)

In [ ]:
submission = pd.DataFrame({'PassengerId': test['PassengerId'],'Survived':tf_prediction.flatten()}) 
submission.to_csv('submission.csv',index=False)

In [ ]:
submission